# Qiskit Runtime REST API query & job submission

## (optional) get temporary Access token from Auth API via API Token

This is especially useful if you would like control over tokens, such as token invalidation. Alternatively, you can work directly with your IBM Quantum Platform API token.

In [2]:
import requests

with open('token') as file:
    token=file.read()

url = 'https://auth.quantum-computing.ibm.com/api/users/loginWithToken'
input={'apiToken': token}
auth_response = requests.post(url, json=input)
auth_id=auth_response.json()['id']

## GET available backends

In [3]:
url_backends = 'https://api.quantum-computing.ibm.com/runtime/backends'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}

backends_response = requests.get(url_backends, headers=headers)

print(backends_response.json()['devices'][:5],"...")

['ibm_brisbane', 'ibm_kyiv', 'ibm_kyoto', 'ibm_cusco', 'ibm_nazca'] ...


## Run a job

### set up qasm circuit

In [4]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[2];
creg c[2];
x q[0];
cx q[0], q[1];
c[0] = measure q[0]; 
c[1] = measure q[1];
'''

In [5]:
from qiskit.circuit.random import random_circuit
from qiskit import QuantumCircuit, qasm2, qasm3
 
random_circ = random_circuit(5, depth=3, seed=42).decompose(reps=3)
qasm_string = qasm3.dumps(random_circ)

In [12]:
qasm_string

'OPENQASM 3.0;\ninclude "stdgates.inc";\nqubit[5] q;\nU(4.381692553882582, -0.9790625982780766, 0.9790625982780766) q[1];\nu3(0, 0, pi/2) q[1];\nu3(pi/2, 0, pi) q[1];\nu3(0, 0, pi/4) q[1];\nu3(pi/2, 0, pi) q[2];\ncx q[0], q[2];\nU(pi, pi/2, pi/2) q[0];\nU(pi/2, 6.0990755645863075, 5.611645507023389) q[0];\nu3(pi/2, 0, pi) q[2];\nu3(0, 0, 4.782381792256834) q[3];\nu3(0, 0, pi/2) q[3];\nu3(0, 0, -pi/2) q[4];\nu1(-pi/2) q[4];\nu2(0, pi) q[4];\nu1(-pi/2) q[4];\nu3(0, 0, pi/2) q[4];\ncx q[4], q[3];\nu3(-3.06500801258003, 0, 0) q[3];\nu3(-3.06500801258003, 0, 0) q[4];\ncx q[4], q[3];\nu3(0, 0, -pi/2) q[3];\nu3(0, 0, -4.782381792256834) q[3];\ncx q[3], q[1];\nu3(0, 0, -pi/4) q[1];\nu3(pi/2, 0, pi) q[1];\nu3(0, 0, -pi/2) q[1];\nU(4.763205750057398, 0.6567560271723107, -0.6567560271723107) q[1];\nU(pi/2, 0, pi) q[3];\nu3(0, 0, -pi/2) q[4];\nu1(pi/2) q[4];\nu2(0, pi) q[4];\nu1(pi/2) q[4];\nu3(0, 0, pi/2) q[4];\nu3(2.0227619311481257, 0, 0) q[4];\ncx q[2], q[4];\nu3(-2.0227619311481257, 0, 0) q[4

## transpile circuit via transpilation service API

Query the cloud Transpilation service at https://cloud-transpiler.quantum-computing.ibm.com/docs . See more details and documentation at https://docs.quantum.ibm.com/transpile/qiskit-transpiler-service . Note that [transpilation is now necessary](https://docs.quantum.ibm.com/announcements/product-updates/2024-02-14-qiskit-runtime-primitives-update) prior to submitting a circuit to IBM Quantum backends. 

In [6]:
backend='ibm_quebec'

In [7]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
        }
body= {
    "qasm_circuits": qasm_string,
}

params = {
    "backend": backend,
    "use_ai": True,
    "optimization_level": 1,
    "coupling_map": [],
    "qiskit_transpile_options": [],
    "ai_layout_mode": 'OPTIMIZE', # 'KEEP', 'OPTIMIZE', 'IMPROVE',
    }



resp = requests.post(
            "https://cloud-transpiler.quantum.ibm.com/transpile",
            headers=headers,
            json=body,
            params = params,
        )


In [ ]:
task_id=resp.json()['task_id']
task_id

### Request the transpilation service results using the task_id

In [9]:
res = requests.get(url=f"https://cloud-transpiler.quantum.ibm.com/transpile/{task_id}", headers=headers)
print(res.json())

{'state': 'SUCCESS', 'result': [{'qasm': 'OPENQASM 3.0; include "stdgates.inc"; gate rzx_140298706869648(_gate_p_0) _gate_q_0, _gate_q_1 {   h _gate_q_1;   cx _gate_q_0, _gate_q_1;   rz(pi/4) _gate_q_1;   cx _gate_q_0, _gate_q_1;   h _gate_q_1; } gate rzx_140298706869968(_gate_p_0) _gate_q_0, _gate_q_1 {   h _gate_q_1;   cx _gate_q_0, _gate_q_1;   rz(-pi/4) _gate_q_1;   cx _gate_q_0, _gate_q_1;   h _gate_q_1; } gate ecr _gate_q_0, _gate_q_1 {   rzx_140298706869648(pi/4) _gate_q_0, _gate_q_1;   x _gate_q_0;   rzx_140298706869968(-pi/4) _gate_q_0, _gate_q_1; } sx $0; rz(-pi/2) $0; x $1; rz(pi/2) $1; ecr $1, $0; rz(-pi/2) $0; sx $0; rz(-2.470052853433595) $0; sx $0; rz(-1.7549060693881753) $0; sx $1; rz(-pi/2) $1; sx $2; rz(-1.500803514922753) $3; sx $3; rz(-pi) $3; ecr $3, $2; sx $2; rz(-3.06500801258003) $2; sx $3; rz(3.06500801258003) $3; sx $3; rz(pi/2) $3; ecr $3, $2; sx $2; rz(0.4519656043532292) $2; sx $2; rz(-pi/2) $2; ecr $2, $1; sx $1; sx $2; rz(2.022761931148125) $2; sx $2; rz(

In [10]:
if res.json().get("state") == "SUCCESS":
    resulting_qasm=res.json().get("result")[0].get("qasm")

print(resulting_qasm)

OPENQASM 3.0; include "stdgates.inc"; gate rzx_140298706869648(_gate_p_0) _gate_q_0, _gate_q_1 {   h _gate_q_1;   cx _gate_q_0, _gate_q_1;   rz(pi/4) _gate_q_1;   cx _gate_q_0, _gate_q_1;   h _gate_q_1; } gate rzx_140298706869968(_gate_p_0) _gate_q_0, _gate_q_1 {   h _gate_q_1;   cx _gate_q_0, _gate_q_1;   rz(-pi/4) _gate_q_1;   cx _gate_q_0, _gate_q_1;   h _gate_q_1; } gate ecr _gate_q_0, _gate_q_1 {   rzx_140298706869648(pi/4) _gate_q_0, _gate_q_1;   x _gate_q_0;   rzx_140298706869968(-pi/4) _gate_q_0, _gate_q_1; } sx $0; rz(-pi/2) $0; x $1; rz(pi/2) $1; ecr $1, $0; rz(-pi/2) $0; sx $0; rz(-2.470052853433595) $0; sx $0; rz(-1.7549060693881753) $0; sx $1; rz(-pi/2) $1; sx $2; rz(-1.500803514922753) $3; sx $3; rz(-pi) $3; ecr $3, $2; sx $2; rz(-3.06500801258003) $2; sx $3; rz(3.06500801258003) $3; sx $3; rz(pi/2) $3; ecr $3, $2; sx $2; rz(0.4519656043532292) $2; sx $2; rz(-pi/2) $2; ecr $2, $1; sx $1; sx $2; rz(2.022761931148125) $2; sx $2; rz(pi/2) $2; ecr $2, $1; rz(pi/2) $1; sx $1; 

### run circuit via API

Note the jobs below use Qiskit Runtime V2 primitives. C.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
Both SamplerV2 and EstimatorV2 take one or more primitive unified blocs (PUBs) as the input. Each PUB is a tuple that contains one circuit and the data broadcasted to that circuit, which can be multiple observables and parameters. Each PUB returns a result.

In [ ]:
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
hub = '<YOUR HUB>'
group = '<YOUR GROUP>'
project = '<YOUR PROJECT>'

headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend, 
    "hub": hub,
    "group": group,
    "project": project,
    "start_session": "False", #set to False if you just need to run a single job.  
    "params": {
        "pubs": [ #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]], 
        "version": 2 #this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

### check job status

In [15]:
response_status_singlejob= requests.get(url+'/'+job_id, headers=headers)
response_status_singlejob.json().get('state')

{'status': 'Completed', 'reason': '', 'reasonCode': None}

### Get job results

In [14]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

res_dict=response_result.json()

estimator_result=res_dict['results']
print(estimator_result)

[{'data': {'evs': 0.5121617512921861, 'stds': 0.2692213349927506, 'ensemble_standard_error': 0.12655983446198443}, 'metadata': {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}}]


In [47]:
exp_val=res_dict['results'][0]['data']['evs']
print('Expectation value: ', exp_val)

Expectation value:  0.7428980350102542


## Exploring Qiskit Runtime Options

In [ ]:
import requests
 
url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
  'Content-Type': 'application/json',
  'x-access-token':auth_id,
  'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' # specifying the application you might be running from. For an actual Integration project, this option is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
  }
job_input = {
  'program_id': 'estimator',
  "backend": backend, 
  "hub": hub,
  "group": group,
  "project": project,
  "start_session": "False", # set to False if you just need to run a single job.  
  "params": { 
      "pubs": [ #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
            [resulting_qasm, # QASM circuit
             {"IIZII": 1, "XIZZZ": 2.3}, # Observable
             None # parameter values
             ]],  
      "version": 2, # this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
      "options": {
          "resilience": {
              "measure_mitigation": True,
              "zne_mitigation": True,
              "zne": {
                  "extrapolator":["exponential", "linear"],
                  "noise_factors":[1, 3, 5],
              },
              #"pec_mitigation": False,
              #"pec": None,
              #"layer_noise_learning":None,
          },
          #"dynamical_decoupling": { 
          #    "enable": True,
          #    "sequence_type": 'XpXm',
          #    "extra_slack_distribution": 'middle',
          #    "scheduling_method": 'alap',
          #},
          #"twirling": { 
          #    "enable_gates": True, 
          #    "enable_measure": True,
          #    "num_randomizations": "auto",
          #    "shots_per_randomization": "auto",
          #    "strategy": "active-accum",
          #    },
              
      },
  }
}
 
response = requests.post(url, headers=headers, json=job_input)
 
if response.status_code == 200:
  job_id = response.json().get('id')
  print("Job created:",response.text)
else:
  print(f"Error: {response.status_code}")

In [ ]:
response_status_singlejob= requests.get(url+'/'+job_id, headers=headers)
print(response_status_singlejob.json().get('state'))

response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

res_dict=response_result.json()

estimator_result=res_dict['results']
print(estimator_result)

## Create Session 

c.f. documentation at https://docs.quantum.ibm.com/api/runtime

In [ ]:
import json
 
sessionsUrl = "https://api.quantum-computing.ibm.com/runtime/sessions"
 
headersList = {
  "Accept": "application/json",
  # "Authorization": "Bearer "+token,
  'x-access-token':auth_id,
  "Content-Type": "application/json" 
}
 
payload = {
  "backend": backend,
  "instance": "ibm-q-internal/dev-sys-software/internal-test",
  "mode": "batch"
}
 
response = requests.request("POST", sessionsUrl, json=payload,  headers=headersList)
 
sessionId = response.json()['id']
 
print(response.json())

### Submit job(s) against resulting Session ID

#### Submit EstimatorV2 job to Session

In [ ]:
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }
job_input = {
    'program_id': 'estimator',
    "backend": backend, 
    "hub": hub,
    "group": group,
    "project": project,
    "session_id": sessionId, # This specifies the previously created Session
    "params": {
        "pubs": [[resulting_qasm,{"II": 1, "ZZ": 2.3},None,0.01]], #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "version": 2, #this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "options": {
            "transpilation":{"optimization_level": 1},
            "resilience": {
                    "measure_mitigation": True, #c.f. documentation at https://docs.quantum.ibm.com/run/configure-error-mitigation#custom-error-settings-v2-primitives 
                #     # "measure_noise_learning": None,
                #     # "zne_mitigation": None,
                #     # "zne": None,
                #     # "pec_mitigation": None,
                #     # "pec": None,
                #     # "layer_noise_learning": None,
                    },
            "twirling": {"enable_gates": True},
            "dynamical_decoupling": {"enable": False},#, "sequence_type": "XpXm"},
                     
                },
        # "resilience_level": 0,        
    }
        # "circuits": [resulting_qasm] #only use this input for Primitives V1, i.e. "version": 1
}
response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id_estimator = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

### Get the Job Results

In [62]:
response_status= requests.get(url+'/'+job_id_estimator, headers=headers)
response_status.json().get('state')


{'status': 'Completed', 'reason': '', 'reasonCode': None}

In [63]:
response_estimator = requests.get(url+'/'+job_id_estimator+'/results', headers=headers)
res_dict_estimator=response_estimator.json()

print(res_dict_estimator['results'])


[{'data': {'evs': -1.3206712251052517, 'stds': 0.019759525689619355, 'ensemble_standard_error': 0.0200795395210333}, 'metadata': {'shots': 10016, 'target_precision': 0.01, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}}]


In [64]:
exp_val=res_dict_estimator['results'][0]['data']['evs']
print('Expectation value: ', exp_val)


Expectation value:  -1.3206712251052517


### close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users. Documentation at https://docs.quantum.ibm.com/run/run-jobs-in-session

In [87]:
closureURL="https://api.quantum-computing.ibm.com/runtime/sessions/"+sessionId+"/close"

headersList = {
  "Accept": "application/json",
  # "Authorization": "Bearer "+token,
  'x-access-token':auth_id,
}
 
closure_response = requests.request(
    "DELETE",
    closureURL, 
    headers=headersList)

print("Session closure response ok?:",closure_response.ok,closure_response.text)

Session closure response ok?: True 


## (Optional) Invalidate Token 

In [93]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)
print("response ok?:",logout_response.ok,logout_response.text)

response ok?: True 


## (Optional) Test Token Invalidation

This should yield an Authoritzation error (Error 401) once access token is invalidated

In [94]:
logout_url = 'https://auth.quantum-computing.ibm.com/api/users/logout'
headers = {'x-access-token':auth_id}
logout_response = requests.post(logout_url, headers=headers)

if logout_response.status_code == 200:
    job_id = logout_response.json().get('id')
    print("Job created:",logout_response.text)
elif logout_response.status_code == 401:
    print("invalid credentials. Access token should be successfully invalidated.")
else:
    print(logout_response.text,"\n")
    print(f"Error: {logout_response.status_code}")

invalid credentials. Access token should be successfully invalidated.
